# Amazon product catalog


## Problem scope

What do Amazon reviews say about the product, and can reviews be used reliably to predict the product category?

## Open questions / workflow

1. Predict the rating based on item desc.: regression w/ language data

2. How well reviewed something is

3. Figure out product, product contents, product tags, document per row + brand + company type, product category, description

4. Probability that the thing we labeled is actually in that class?


## Data imports

In [1]:
#libraries
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, re

### Product dataset 1

In [2]:
#import data from mktg dataset
#from https://stackoverflow.com/questions/9652832/how-to-load-a-tsv-file-into-a-pandas-dataframe
#read_csv('path_to_file', sep='\t')
products = pd.read_csv(
    '../data/marketing_sample_for_amazon_com-amazon_product_data__20200401_20200630__30k_data.tsv',
sep='\t')

In [3]:
#peek
products.head(2)

,Uniq Id,Crawl Timestamp,Dataset Origin,Product Id,Product Barcode,Product Company Type Source,Product Brand Source,Product Brand Normalised Source,Product Name Source,Match Rank,...,Product Currency,Product Available Inventory,Product Image Url,Product Model Number,Product Tags,Product Contents,Product Rating,Product Reviews Count,Bsr,Joining Key
0,690e47123b04e15df768a9f54f1654e8,2020-05-13 11:02:04 +0000,NaN,ed177532df52b8b2fdba6cff731b9d00,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NaN,100% Linen Table Runner 14x90 Tailored with Mi...,100% Pure Linen Imported 100% Pure Linen table...,3.5,4,"#83,321 in Kitchen & Dining (See Top 100 in Ki...",78af1f66f1e365647484688360621e5e
1,b02a91cfdae2c5596b568e771b402176,2020-06-27 15:31:29 +0000,NaN,8756f065784790599a218174bc932395,UPC: 799460760240,Competitor,NaN,NaN,NaN,NaN,...,USD,999999999,https://images-na.ssl-images-amazon.com/images...,NL-F-5-2,2 pcs/set LONG Home Sauna Spa Exfoliating Nylo...,The nylon exfoliating cloth is the best bath p...,0.0,0,"#175,862 in Beauty & Personal Care (See Top 10...",83e8d81095fe218d5bb580194669e0c4


In [4]:
products.columns

Index(['Uniq Id', 'Crawl Timestamp', 'Dataset Origin', 'Product Id',
       'Product Barcode', 'Product Company Type Source',
       'Product Brand Source', 'Product Brand Normalised Source',
       'Product Name Source', 'Match Rank', 'Match Score', 'Match Type',
       'Retailer', 'Product Category', 'Product Brand', 'Product Name',
       'Product Price', 'Sku', 'Upc', 'Product Url', 'Market',
       'Product Description', 'Product Currency',
       'Product Available Inventory', 'Product Image Url',
       'Product Model Number', 'Product Tags', 'Product Contents',
       'Product Rating', 'Product Reviews Count', 'Bsr', 'Joining Key'],
      dtype='object')

In [5]:
products.shape

(29984, 32)

In [6]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29984 entries, 0 to 29983
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Uniq Id                          29984 non-null  object 
 1   Crawl Timestamp                  29984 non-null  object 
 2   Dataset Origin                   0 non-null      float64
 3   Product Id                       29984 non-null  object 
 4   Product Barcode                  6291 non-null   object 
 5   Product Company Type Source      29984 non-null  object 
 6   Product Brand Source             71 non-null     object 
 7   Product Brand Normalised Source  71 non-null     object 
 8   Product Name Source              71 non-null     object 
 9   Match Rank                       0 non-null      float64
 10  Match Score                      0 non-null      float64
 11  Match Type                       0 non-null      float64
 12  Retailer          

In [7]:
products.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
Dataset Origin,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Match Rank,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Match Score,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Match Type,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Product Price,23553.0,4.129000e+01,1.211600e+02,1.000000e-02,1.129000e+01,18.5,3.499000e+01,7400.0
Product Available Inventory,29984.0,9.099964e+08,2.681509e+08,1.111111e+08,1.000000e+09,999999999.0,1.000000e+09,999999999.0
Product Rating,29984.0,3.120000e+00,1.980000e+00,0.000000e+00,0.000000e+00,4.1,4.600000e+00,5.0
Product Reviews Count,29984.0,5.356000e+01,3.419800e+02,0.000000e+00,0.000000e+00,3.0,2.000000e+01,24231.0


### Product dataset 2

In [8]:
#import ecom dataset
ecom = pd.read_csv('../data/amazon_co-ecommerce_sample.csv')

In [9]:
ecom.shape

(10000, 17)

In [10]:
ecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 9993 non-null   object 
 3   price                                        8565 non-null   object 
 4   number_available_in_stock                    7500 non-null   object 
 5   number_of_reviews                            9982 non-null   object 
 6   number_of_answered_questions                 9235 non-null   float64
 7   average_review_rating                        9982 non-null   object 
 8   amazon_category_and_sub_category             9310 non-null   object 
 9   customers_who_bought_this_item_also_bought   8938 non-null   object 
 10 

In [11]:
ecom.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


## Cleaning

Let's clean up our columns, including price, so that they can converge properly.

### Remove extraneous characters

From the pound signs in the prices, to prices listed as ranges, we have some issues in our data formats. Let's fix these.

For price ranges, I will merely consume the lower bound of the price.

In [14]:
#Substantial datatype conversion help here from https://pbpython.com/currency-cleanup.html

def clean_price(x):
    
    if isinstance(x, str):
#If the value is a string, then remove currency symbol, delimiters and anything 
#else following the price; otherwise, the value is numeric and can be converted as is.
        
        return(x.replace('£', '').replace(',', '').split(' - ')[0])
    #strip price of the pound sign
    #we need to get rid of price ranges. I will just consume the LOWER BOUND in case of a price range.

    return(x)

In [15]:
#apply my function
ecom['price'] = ecom['price'].apply(clean_price).astype('float')

In [ ]:
#strip out "out of 5 stars" from average_review_rating
ecom['average_review_rating'] = ecom['average_review_rating'].str.strip(' out of 5 stars')
ecom['average_review_rating'].head(2)

In [ ]:
#strip out the word "new" from number in stock -- but not actually useful for training, so can skip this?

In [ ]:
#strip out commas from number_of_reviews column
ecom['number_of_reviews'] = ecom['number_of_reviews'].str.strip(",")
ecom['number_of_reviews'].sort_values(ascending=False)[:30]

### Converting object columns to numeric where applicable

In [ ]:
#force num type on number_of_reviews, number_of_answered_questions, price, average_review_rating

ecom['number_of_answered_questions'] = ecom[['number_of_answered_questions']].apply(pd.to_numeric)

In [ ]:
ecom['average_review_rating'] = ecom[['average_review_rating']].apply(pd.to_numeric)

In [ ]:
ecom['number_of_reviews'] = ecom[['number_of_reviews']].apply(pd.to_numeric)

In [16]:
#confirm conversions
ecom.dtypes

uniq_id                                         object
product_name                                    object
manufacturer                                    object
price                                          float64
number_available_in_stock                       object
number_of_reviews                               object
number_of_answered_questions                   float64
average_review_rating                           object
amazon_category_and_sub_category                object
customers_who_bought_this_item_also_bought      object
description                                     object
product_information                             object
product_description                             object
items_customers_buy_after_viewing_this_item     object
customer_questions_and_answers                  object
customer_reviews                                object
sellers                                         object
dtype: object

### Imputing nulls

For most of the fields, imputing with a 0 seems to make sense -- no reviews is no different than 0 reviews.

For price, however, we'll want to impute with the mean, now that we've successfully converted it to numeric.

As for average review rating, in the absence of one, we will simply drop those few observations since that's a valuable column and I'd want to be careful extending any kind of mean to it, since it is possibly our target.

In [ ]:
#`dropna.()` nulls without an average_review_rating since we need those for training, and there are few
ecom.dropna(subset=['average_review_rating'], inplace=True)

In [ ]:
#fill price with average of column
ecom['price'] = ecom['price'].fillna(ecom['price'].mean())

In [ ]:
#fill in NaN's with 0's for everything else
ecom = ecom.fillna(0)
ecom.head(3)

In [ ]:
ecom.isnull().sum()

## EDA

In [ ]:
ecom.columns

In [ ]:
ecom['price'].value_counts().sort_values(ascending=False)[:15]

In [13]:
ecom['price'].max()

TypeError: '>=' not supported between instances of 'str' and 'float'

In [ ]:
ecom.describe()

In [ ]:
ecom['price'].mean()

In [ ]:
ecom['number_available_in_stock'].value_counts()[:10]

In [ ]:
ecom['number_of_answered_questions'].mean()

In [ ]:
ecom['number_of_reviews'].mean()

## Feature engineering

We need to convert things like ratings so that the modeling works correctly.

The plan:

* Categorize / one hot encode the ratings column 

In [ ]:
#one-hot encode ratings column

In [ ]:
#move out new vs. used items into sep. columns (binarize)

## NLP

In [ ]:
#vectorize text

## Modeling - classifier

In [ ]:
#reviews
#price
#category

## Evaluation

## Conclusions & next steps